## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-08-02-02-00 +0000,wsj,New Zealand’s Central Bank Delivers Outsized C...,https://www.wsj.com/articles/new-zealands-cent...
1,2025-10-08-02-00-51 +0000,nyt,Trump-Backed Republican Prevails in Tennessee ...,https://www.nytimes.com/2025/10/07/us/politics...
2,2025-10-08-01-58-59 +0000,nypost,Toyota recalls nearly 394K vehicles over rearv...,https://nypost.com/2025/10/07/business/toyota-...
3,2025-10-08-01-52-22 +0000,startribune,Wolves’ Bones Hyland has a supporter in Tim Co...,https://www.startribune.com/minnesota-timberwo...
4,2025-10-08-01-48-34 +0000,latimes,City seeks to overturn judge's order limiting ...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,36
0,new,16
253,gaza,13
118,shutdown,12
403,will,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
165,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...,91
174,2025-10-07-17-41-13 +0000,nypost,Trump says Dems ‘have no leader’ in shutdown s...,https://nypost.com/2025/10/07/us-news/trump-ri...,82
310,2025-10-07-08-00-00 +0000,cbc,Trump says 'Canada will love us again' — but t...,https://www.cbc.ca/news/politics/trump-carney-...,82
45,2025-10-07-23-53-21 +0000,nypost,Mother of US Navy medic rescued from Gaza in d...,https://nypost.com/2025/10/07/world-news/mothe...,80
90,2025-10-07-21-52-46 +0000,nyt,Trump Suggests Furloughed Federal Workers May ...,https://www.nytimes.com/2025/10/07/us/politics...,76


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
165,91,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...
107,55,2025-10-07-21-10-44 +0000,latimes,Sensitive police files to be opened up for wat...,https://www.latimes.com/california/story/2025-...
115,55,2025-10-07-21-00-48 +0000,nypost,Tariffs will feed 7 million mothers and babies...,https://nypost.com/2025/10/07/us-news/tariffs-...
186,51,2025-10-07-17-01-48 +0000,latimes,Supreme Court sees a free-speech problem with ...,https://www.latimes.com/politics/story/2025-10...
121,46,2025-10-07-20-52-33 +0000,nyt,Israel Marks Two-Year Anniversary of Oct. 7 in...,https://www.nytimes.com/2025/10/07/world/middl...
222,41,2025-10-07-15-03-14 +0000,nypost,Nobel winner Fred Ramsdell didn’t know he’d wo...,https://nypost.com/2025/10/07/world-news/nobel...
208,40,2025-10-07-15-50-55 +0000,nypost,NY woman refuses to pay $440 fine to Hertz aft...,https://nypost.com/2025/10/07/business/ny-woma...
133,39,2025-10-07-20-26-13 +0000,nypost,Connecticut governor warns Wall Street at risk...,https://nypost.com/2025/10/07/business/connect...
204,37,2025-10-07-16-03-10 +0000,nypost,Pam Bondi lashes out at Dems in rowdy hearing ...,https://nypost.com/2025/10/07/us-news/pam-bond...
8,36,2025-10-08-01-21-59 +0000,nypost,Defunct marine park threatens to kill 30 belug...,https://nypost.com/2025/10/07/world-news/canad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
